In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

AttributeError: module 'numpy' has no attribute 'scipy.stats._boost'

In [3]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/Song Dataset.csv',sep=",", encoding='cp1252')

In [4]:
data.head(5)

,title,singer,genre,language
0,All of the Stars,Ed Sheeran,Pop,English
1,Beautiful,Christina Aguilera,Pop,English
2,Born This Way,Lady Gaga,Pop,English
3,Don't Stop Believin',Journey,Rock,English
4,FireFly,Ed Sheeran,Pop,English


In [5]:
data.dtypes

title       object
singer      object
genre       object
language    object
dtype: object

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     79 non-null     object
 1   singer    79 non-null     object
 2   genre     79 non-null     object
 3   language  79 non-null     object
dtypes: object(4)
memory usage: 2.6+ KB


In [7]:
data['genre'] = data['genre'].astype('str')
data['genre'] = data['genre'].str.split() 

In [8]:
data.head(10)

,title,singer,genre,language
0,All of the Stars,Ed Sheeran,[Pop],English
1,Beautiful,Christina Aguilera,[Pop],English
2,Born This Way,Lady Gaga,[Pop],English
3,Don't Stop Believin',Journey,[Rock],English
4,FireFly,Ed Sheeran,[Pop],English
5,Fooling Yourself,Styx,[Rock],English
6,GIRL,Maren Morris,"[Country, Pop]",English
7,Grand Illusion,Styx,[Rock],English
8,He Will Hold Me Fast,Keith & Kristyn Getty,"[Christian, Music]",English
9,Hero,Mariah Carey,"[Dance, Pop]",English


In [9]:
s = data.apply(lambda x: pd.Series(x['genre']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
data=data.drop('genre', axis=1).join(s)

In [10]:
data.head(10)

,title,singer,language,genre
0,All of the Stars,Ed Sheeran,English,Pop
1,Beautiful,Christina Aguilera,English,Pop
2,Born This Way,Lady Gaga,English,Pop
3,Don't Stop Believin',Journey,English,Rock
4,FireFly,Ed Sheeran,English,Pop
5,Fooling Yourself,Styx,English,Rock
6,GIRL,Maren Morris,English,Country
6,GIRL,Maren Morris,English,Pop
7,Grand Illusion,Styx,English,Rock
8,He Will Hold Me Fast,Keith & Kristyn Getty,English,Christian


In [11]:
data['index']=data.index
data.head(10)

,title,singer,language,genre,index
0,All of the Stars,Ed Sheeran,English,Pop,0
1,Beautiful,Christina Aguilera,English,Pop,1
2,Born This Way,Lady Gaga,English,Pop,2
3,Don't Stop Believin',Journey,English,Rock,3
4,FireFly,Ed Sheeran,English,Pop,4
5,Fooling Yourself,Styx,English,Rock,5
6,GIRL,Maren Morris,English,Country,6
6,GIRL,Maren Morris,English,Pop,6
7,Grand Illusion,Styx,English,Rock,7
8,He Will Hold Me Fast,Keith & Kristyn Getty,English,Christian,8


In [12]:
data = pd.DataFrame(data, index=data['index'])
data.reset_index(drop=True,inplace=True)

In [13]:
data.shape

(121, 5)

In [14]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['genre'])
tfidf_matrix.shape

(121, 26)

In [15]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
indices = pd.Series(data.index, index=data['genre'])

In [17]:
def get_recommendations(genre,cosine_sim=cosine_sim,num=10):
    idx=indices[genre]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=list(np.argsort(sim_scores[0][1]))
    sim_scores.reverse()
    top_similar=sim_scores[0:num+1]
    song_idx=[i for i in top_similar]
    ans=data[['title','singer']].iloc[song_idx]
    ans=ans.sample(n=10)
    return ans

In [18]:
get_recommendations('Soft')

,title,singer
94,Uptown Girl,Billy Joel
40,Wonder,Natalie Merchant
34,This is It,Kenny Loggins
13,Higher Power,Boston
92,I'm A Believer,The Monkees
120,Zindagi Hosh Mein,KK and Zubeen Garg
22,I Won't Back Down,Tom Petty and the Heartbreakers
38,When the Seasons Change,Five Finger Death Punch
37,Unwell,Matchbox Twenty
39,Wonder,Natalie Merchant


In [19]:
def get_recommendation(genre):
    df=data[data['genre']==genre]
    ans=df[['title','singer']]
    if(len(ans)<10):
      return ans
    else:
      return ans.sample(n=10)

In [21]:
get_recommendation('Soft')

,title,singer
13,Higher Power,Boston
22,I Won't Back Down,Tom Petty and the Heartbreakers
34,This is It,Kenny Loggins
92,I'm A Believer,The Monkees
94,Uptown Girl,Billy Joel
96,Eye Of The Tiger,Survivor


In [22]:
import pickle
pickle.dump(data,open('/content/drive/My Drive/Colab Notebooks/pkl/song_list.pkl','wb'))
pickle.dump(cosine_sim,open('/content/drive/My Drive/Colab Notebooks/pkl/cosine_sim_song.pkl','wb'))